1) Setup


In [1]:
!pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip -q install scikit-learn pandas tqdm iterative-stratification


In [2]:
import os, json, math, random, gc, time
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import torchvision
from torchvision import transforms
from PIL import Image


2) Paths & classes

In [3]:
# ajuste este caminho para sua máquina/Colab
ROOT = Path('NIH_ChestXray14')  # ex: '/content/drive/MyDrive/NIH_ChestXray14'
CSV_PATH = ROOT / 'Data_Entry_2017.csv'
IMAGES_DIR = ROOT / 'images'

# Classes exigidas no enunciado, respeitando o "Nodule Mass" combinado
TARGETS = [
    'Atelectasis','Consolidation','Infiltration','Pneumothorax','Edema',
    'Emphysema','Fibrosis','Effusion','Pneumonia','Pleural_thickening',
    'Cardiomegaly','Nodule Mass','Hernia'
]

IMG_EXTS = {'.png', '.jpg', '.jpeg'}
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)


3) Carregar CSV e filtrar apenas pelas imagens existentes

In [4]:
df = pd.read_csv(CSV_PATH)

# normaliza nomes de colunas principais do NIH
df = df.rename(columns={'Image Index':'image_name', 'Finding Labels':'labels'})

# imagens realmente presentes na pasta
available = {
    p.name for p in IMAGES_DIR.iterdir()
    if p.is_file() and p.suffix.lower() in IMG_EXTS
}

# filtra df
df = df[df['image_name'].isin(available)].reset_index(drop=True)
len(df), df.head(3)


(4193,
          image_name              labels  Follow-up #  Patient ID  Patient Age  \
 0  00000028_000.png  Pleural_Thickening            0          28           63   
 1  00000029_000.png          No Finding            0          29           59   
 2  00000030_000.png         Atelectasis            0          30           74   
 
   Patient Gender View Position  OriginalImage[Width  Height]  \
 0              M            PA                 2048     2500   
 1              F            PA                 2500     2048   
 2              M            PA                 2992     2991   
 
    OriginalImagePixelSpacing[x     y]  Unnamed: 11  
 0                        0.168  0.168          NaN  
 1                        0.171  0.171          NaN  
 2                        0.143  0.143          NaN  )

4) Parse dos rótulos e mapa para as 13 classes (combinando “Nodule” e “Mass”)

In [5]:
def parse_labels(s: str):
    # 'No Finding' => vetor zero
    if s == 'No Finding':
        return set()
    return set([x.strip() for x in s.split('|') if x.strip()])

def row_to_multihot(orig_set: set):
    # mapping 1:1 exceto "Nodule Mass" := (Nodule OR Mass)
    vals = []
    for c in TARGETS:
        if c == 'Nodule Mass':
            vals.append( int(('Nodule' in orig_set) or ('Mass' in orig_set)) )
        elif c == 'Pleural_thickening':
            # NIH usa 'Pleural_Thickening' (T maiúsculo às vezes). Normaliza:
            vals.append( int(('Pleural_Thickening' in orig_set) or ('Pleural_thickening' in orig_set)) )
        else:
            vals.append( int(c in orig_set) )
    return vals

df['label_set'] = df['labels'].map(parse_labels)
Y = np.vstack(df['label_set'].map(row_to_multihot).values)
Y.shape


(4193, 13)

5) Split multirrótulo estratificado (train/val/test)

In [6]:
# proporções (ajuste se quiser)
train_prop, val_prop, test_prop = 0.7, 0.15, 0.15
n_splits = int(1/val_prop)  # aproxima para pegar uma fold ~ val_prop

mskf = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)

# 1º split: train vs val-candidate
idx = np.arange(len(df))
train_idx, val_idx = next(mskf.split(idx, Y))

X_train, Y_train = idx[train_idx], Y[train_idx]
X_valcand, Y_valcand = idx[val_idx], Y[val_idx]

# 2º split do "val-candidate": val vs test
mskf2 = MultilabelStratifiedKFold(n_splits=2, shuffle=True, random_state=SEED)
val_idx2, test_idx2 = next(mskf2.split(X_valcand, Y_valcand))

val_idx_final  = X_valcand[val_idx2]
test_idx_final = X_valcand[test_idx2]

len(X_train), len(val_idx_final), len(test_idx_final)


(3494, 350, 349)

6) Dataset & DataLoader (com augmentations leves e 3 canais)

In [7]:
IMG_SIZE = 224
BATCH = 32
NUM_WORKERS = 2

train_tfms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),   # X-ray mono → 3 canais
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]) # ImageNet stats
])

val_tfms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

class ChestXrayDataset(Dataset):
    def __init__(self, df, indices, targets, img_dir, tfm):
        self.df = df.reset_index(drop=True)
        self.indices = indices
        self.targets = targets
        self.img_dir = Path(img_dir)
        self.tfm = tfm
    def __len__(self): return len(self.indices)
    def __getitem__(self, i):
        ridx = self.indices[i]
        row = self.df.iloc[ridx]
        img = Image.open(self.img_dir/row['image_name']).convert('L') # robustez
        img = self.tfm(img)
        y = torch.tensor(self.targets[ridx], dtype=torch.float32)
        return img, y

train_ds = ChestXrayDataset(df, X_train, Y, IMAGES_DIR, train_tfms)
val_ds   = ChestXrayDataset(df, val_idx_final, Y, IMAGES_DIR, val_tfms)
test_ds  = ChestXrayDataset(df, test_idx_final, Y, IMAGES_DIR, val_tfms)

train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_dl  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)


7) Modelos: Teacher = ResNet50, Student = MobileNetV2

In [8]:
NUM_CLASSES = len(TARGETS)

def build_resnet50(num_classes=NUM_CLASSES, pretrained=True):
    m = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V2 if pretrained else None)
    in_features = m.fc.in_features
    m.fc = nn.Linear(in_features, num_classes)
    return m

def build_mobilenet_v2(num_classes=NUM_CLASSES, pretrained=True):
    m = torchvision.models.mobilenet_v2(weights=torchvision.models.MobileNet_V2_Weights.IMAGENET1K_V2 if pretrained else None)
    in_features = m.classifier[-1].in_features
    m.classifier[-1] = nn.Linear(in_features, num_classes)
    return m

teacher = build_resnet50().to(DEVICE)
student = build_mobilenet_v2().to(DEVICE)


9) Knowledge Distillation (KL com temperatura + BCE dura)

In [9]:
# 0) instalar
!pip -q install torchxrayvision

# 1) carregar o teacher pronto (NIH, 224x224)
import torchxrayvision as xrv

# densenet NIH; pode usar também: "densenet121-res224-all" ou "resnet50-res512-all"
teacher_raw = xrv.models.DenseNet(
    weights="densenet121-res224-nih",
    apply_sigmoid=False  # queremos LOGITS p/ KD
).to(DEVICE).eval()

# 2) checar as saídas do modelo (ordem das patologias)
print(teacher_raw.targets)
# ['Atelectasis','Consolidation','Infiltration','Pneumothorax','Edema','Emphysema','Fibrosis',
#  'Effusion','Pneumonia','Pleural_Thickening','Cardiomegaly','Nodule','Mass','Hernia',
#  'Lung Lesion','Fracture','Lung Opacity','Enlarged Cardiomediastinum']

# 3) criar um wrapper que mapeia as 18 saídas do teacher → suas 13 TARGETS
import torch
import torch.nn as nn

TARGETS = [
    'Atelectasis','Consolidation','Infiltration','Pneumothorax','Edema',
    'Emphysema','Fibrosis','Effusion','Pneumonia','Pleural_thickening',
    'Cardiomegaly','Nodule Mass','Hernia'
]

# mapa de índices do teacher (18) para as suas 13
idx_map = { name:i for i,name in enumerate(teacher_raw.targets) }

sel_indices = [
    idx_map['Atelectasis'],
    idx_map['Consolidation'],
    idx_map['Infiltration'],
    idx_map['Pneumothorax'],
    idx_map['Edema'],
    idx_map['Emphysema'],
    idx_map['Fibrosis'],
    idx_map['Effusion'],
    idx_map['Pneumonia'],
    idx_map['Pleural_Thickening'],  # seu alvo chama Pleural_thickening (normalizamos no parse)
    idx_map['Cardiomegaly'],
    # "Nodule Mass" = OR(Nodule, Mass) nos LOGITS: aprox. via log-sum-exp dos dois
    (idx_map['Nodule'], idx_map['Mass']),
    idx_map['Hernia']
]

class Teacher13(nn.Module):
    def __init__(self, base):
        super().__init__()
        self.base = base
    def forward(self, x):
        logits18 = self.base(x)  # [B,18] (LOGITS se apply_sigmoid=False)
        outs = []
        for item in sel_indices:
            if isinstance(item, tuple):
                # combinar Nodule/Mass em um único logit via log-sum-exp(sigmoid^-1)
                # logits → probs p1,p2 ; OR: p_or = 1 - (1-p1)(1-p2)
                p = torch.sigmoid(logits18[:, item[0]]), torch.sigmoid(logits18[:, item[1]])
                p_or = 1 - (1 - p[0]) * (1 - p[1])
                # volta a "logit" p/ KD: log(p/(1-p))
                logit_or = torch.log(p_or/(1-p_or+1e-8) + 1e-8)
                outs.append(logit_or.unsqueeze(1))
            else:
                outs.append(logits18[:, item].unsqueeze(1))
        return torch.cat(outs, dim=1)  # [B,13]

teacher = Teacher13(teacher_raw).to(DEVICE).eval()
for p in teacher.parameters():
    p.requires_grad_(False)  # congela o teacher


['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Mass', 'Hernia', '', '', '', '']


10) Avaliação final no test set

In [12]:
student.load_state_dict(torch.load('student_best.pt', map_location=DEVICE))
y_true, y_prob, aucs, mauc = evaluate(student, test_dl)

print('mAUROC (test):', round(mauc,4))
for c, a in zip(TARGETS, aucs):
    print(f'{c:>20s}: {a:.4f}' if not np.isnan(a) else f'{c:>20s}: N/A (sem positivos no split)')


FileNotFoundError: [Errno 2] No such file or directory: 'student_best.pt'

11) Inferência em novas imagens

In [ ]:
@torch.no_grad()
def predict_image(path, model, threshold=0.5):
    model.eval()
    img = Image.open(path).convert('L')
    x = val_tfms(img).unsqueeze(0).to(DEVICE)
    logits = model(x)
    prob = torch.sigmoid(logits).cpu().numpy()[0]
    pred = (prob >= threshold).astype(int)
    return {cls: float(p) for cls, p in zip(TARGETS, prob)}, {cls: int(v) for cls, v in zip(TARGETS, pred)}

# exemplo
probs, preds = predict_image(IMAGES_DIR/'00000001_000.png', student, threshold=0.5)
probs, preds


# Knowledge Distillation


## Setup

In [1]:
import os

import keras
from keras import layers
from keras import ops
import numpy as np

In [33]:
import os, glob
from pathlib import Path

ROOT = Path('NIH_ChestXray14')  # ajuste se estiver no Colab/local
IMG_DIRS = [ROOT/'imagens']     # pode colocar várias pastas aqui

valid_exts = {'.png', '.jpg', '.jpeg'}
available_files = set()

for d in IMG_DIRS:
    for p in d.glob('*'):
        if p.suffix.lower() in valid_exts:
            available_files.add(p.name)

len(available_files)


1111

In [34]:
import pandas as pd

CSV_PATH = ROOT/'Data_Entry_2017.csv'  # ajuste o caminho
df = pd.read_csv(CSV_PATH)

# coluna do NIH: 'Image Index' (cada linha = uma imagem)
df_sub = df[df['Image Index'].isin(available_files)].copy()
print(df.shape, '->', df_sub.shape)


(112120, 12) -> (1111, 12)


In [2]:
#import kagglehub

# Download latest version
#path = kagglehub.dataset_download("nih-chest-xrays/data")

#print("Path to dataset files:", path)

SyntaxError: invalid syntax (ipython-input-1972064603.py, line 3)

## Construindo a classe `Distiller()`



In [18]:
# ==== Distiller para Multi-Label (BCE + KD com temperatura) ====
import tensorflow as tf
from tensorflow import keras
from keras import ops

class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(self, optimizer, metrics,
                student_loss_fn, distillation_loss_fn,
                alpha=0.5, temperature=3.0):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        x, y = data
        # Forward teacher (congelado)
        teacher_pred = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            student_pred = self.student(x, training=True)

            # Student supervised loss (BCE multi-label)
            student_loss = self.student_loss_fn(y, student_pred)

            # Distillation loss: KL entre probabilidades suavizadas
            t = self.temperature
            teacher_soft = ops.sigmoid(teacher_pred / t)
            student_soft = ops.sigmoid(student_pred / t)
            distill_loss = self.distillation_loss_fn(teacher_soft, student_soft) * (t**2)

            loss = self.alpha * student_loss + (1.0 - self.alpha) * distill_loss

        grads = tape.gradient(loss, self.student.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.student.trainable_variables))

        # Atualiza métricas no espaço multi-label
        for m in self.metrics:
            m.update_state(y, ops.sigmoid(student_pred))
        return {"loss": loss, "student_loss": student_loss, "distill_loss": distill_loss, **{m.name: m.result() for m in self.metrics}}

    def test_step(self, data):
        x, y = data
        y_pred = self.student(x, training=False)
        s_loss = self.student_loss_fn(y, y_pred)
        for m in self.metrics:
            m.update_state(y, ops.sigmoid(y_pred))
        return {"loss": s_loss, **{m.name: m.result() for m in self.metrics}}

    def call(self, x):
        return self.student(x)


## Criando modelos student and teacher


In [28]:
# ==== Modelos: teacher grande (ResNet50) e student menor (MobileNetV2) ====
from tensorflow import keras
from tensorflow.keras import layers

def build_teacher(input_shape=(224,224,3), num_classes=NUM_CLASSES):
    base = keras.applications.ResNet50(include_top=False, weights="imagenet", input_shape=input_shape)
    base.trainable = True  # pode usar fine-tuning
    inputs = keras.Input(shape=input_shape)
    x = keras.applications.resnet50.preprocess_input(inputs)
    x = base(x, training=True)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    logits = layers.Dense(num_classes, name="logits")(x)  # sem ativação
    return keras.Model(inputs, logits, name="teacher_resnet50")

def build_student(input_shape=(224,224,3), num_classes=NUM_CLASSES):
    base = keras.applications.MobileNetV2(include_top=False, weights="imagenet", input_shape=input_shape)
    base.trainable = True
    inputs = keras.Input(shape=input_shape)
    x = keras.applications.mobilenet_v2.preprocess_input(inputs)
    x = base(x, training=True)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    logits = layers.Dense(num_classes, name="logits")(x)  # sem ativação
    return keras.Model(inputs, logits, name="student_mobilenetv2")

teacher = build_teacher()
student = build_student()

# clone p/ comparação do student treinado "from scratch"
student_scratch = keras.models.clone_model(student)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Preparando o dataset

[MNIST](https://keras.io/api/datasets/mnist/)

In [26]:
# ==== NIH Chest X-ray 14: preparação de dados (multi-label) ====
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

# Caminhos (ajuste para a sua máquina)
BASE_DIR = "NIH_ChestXray14"   # <- ajuste
CSV_PATH = f"{BASE_DIR}/Data_Entry_2017.csv"
IMG_DIR  = f"{BASE_DIR}/images"      # as imagens ficam aqui

# As 13 doenças exigidas (notar que o dataset original tem 'Mass' e 'Nodule' separados)
TARGETS_13 = [
    "Atelectasis","Consolidation","Infiltration","Pneumothorax","Edema",
    "Emphysema","Fibrosis","Effusion","Pneumonia","Pleural_Thickening",
    "Cardiomegaly","Nodule Mass","Hernia"
]

# Observação importante:
# O NIH tem 'Mass' e 'Nodule' como rótulos distintos. O enunciado pede "Nodule Mass" (junto).
# A regra abaixo unifica: se houver 'Nodule' OU 'Mass', marcamos 'Nodule Mass'.
# (Se você preferir manter separados, basta tratar como 14 classes e remover esta unificação.)

df = pd.read_csv(CSV_PATH, encoding='latin-1')
df.rename(columns={"Finding Labels":"Finding_Labels", "Image Index":"Image"}, inplace=True)

# Filtra apenas imagens que tenham ao menos um rótulo das 13 categorias (ignorando 'No Finding')
def normalize_label_string(s):
    return [x.strip() for x in s.split('|') if x.strip()]

def to_multihot_13(labels):
    labels = set(labels)
    # unificação Nodule/Mass:
    has_nodule_mass = ('Nodule' in labels) or ('Mass' in labels)

    hot = {k:0 for k in TARGETS_13}
    for d in labels:
        if d in ["Atelectasis","Consolidation","Infiltration","Pneumothorax","Edema",
                 "Emphysema","Fibrosis","Effusion","Pneumonia","Pleural_Thickening",
                 "Cardiomegaly","Hernia"]:
            hot[d] = 1
    if has_nodule_mass:
        hot["Nodule Mass"] = 1
    return np.array([hot[k] for k in TARGETS_13], dtype=np.float32)

df["labels_list"] = df["Finding_Labels"].apply(normalize_label_string)
df = df[df["labels_list"].map(lambda L: L and L != ["No Finding"])].copy()
df["y"] = df["labels_list"].apply(to_multihot_13)

# split
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42, shuffle=True)

IMG_SIZE = (224, 224)   # redes pré-treinadas tipicamente usam 224x224
BATCH = 32
AUTOTUNE = tf.data.AUTOTUNE

def load_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=1)       # imagens são em escala de cinza
    img = tf.image.grayscale_to_rgb(img)              # repetir canal p/ 3 canais
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    return img

def make_ds(dataframe, training=True):
    paths = tf.constant([f"{IMG_DIR}/{fn}" for fn in dataframe["Image"].tolist()])
    labels = tf.stack(list(dataframe["y"].values))
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    def _map(p, y):
        x = load_image(p)
        if training:
            # augment leve (opcional)
            x = tf.image.random_flip_left_right(x)
        return x, y
    ds = ds.shuffle(4096) if training else ds
    ds = ds.map(_map, num_parallel_calls=AUTOTUNE).batch(BATCH).prefetch(AUTOTUNE)
    return ds

train_ds = make_ds(train_df, training=True)
val_ds   = make_ds(val_df, training=False)

NUM_CLASSES = len(TARGETS_13)

In [27]:
NUM_CLASSES = len(TARGETS_13)

## Treinando o teacher



In [29]:
# ==== Treino do TEACHER (multi-label) ====
from tensorflow.keras.metrics import AUC

teacher.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[AUC(curve="ROC", multi_label=True, num_labels=NUM_CLASSES, name="auroc")],
)

teacher.fit(train_ds, validation_data=val_ds, epochs=5)   # aumente épocas se tiver GPU/tempo
teacher.evaluate(val_ds)


Epoch 1/5


NotFoundError: Graph execution error:

Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) NOT_FOUND:  NIH_ChestXray14/images/00009302_015.png; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_2]]
  (1) NOT_FOUND:  NIH_ChestXray14/images/00009302_015.png; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_multi_step_on_iterator_49690]

## Distill teacher para student



In [ ]:
# ==== Distillation: teacher -> student ====
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    metrics=[AUC(curve="ROC", multi_label=True, num_labels=NUM_CLASSES, name="auroc")],
    student_loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.5,          # 0.3–0.7 costuma funcionar bem
    temperature=3.0,    # 2–5
)
distiller.fit(train_ds, validation_data=val_ds, epochs=5)
distiller.evaluate(val_ds)


## Treinando o student 'from scratch' para comparação

We can also train an equivalent student model from scratch without the teacher, in order
to evaluate the performance gain obtained by knowledge distillation.

In [ ]:
# ==== Student do zero (baseline p/ comparação) ====
student_scratch.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[AUC(curve="ROC", multi_label=True, num_labels=NUM_CLASSES, name="auroc")],
)
student_scratch.fit(train_ds, validation_data=val_ds, epochs=5)
student_scratch.evaluate(val_ds)


Faça a Destilação de um modelo grande pré-treinado, exemplo (VGG, ResNet, ou treinado do zero) para um modelo menor aplicado na resolução do seguinte problema: https://www.kaggle.com/datasets/nih-chest-xrays/data

Neste problema, o seu modelo deverá indicar para cada imagem de RaioX do tórax uma das seguintes doenças(Utilize a coluna Finding Labels do arquivo Data_Entry_2017.csv como variável target):

Atelectasis
Consolidation
Infiltration
Pneumothorax
Edema
Emphysema
Fibrosis
Effusion
Pneumonia
Pleural_thickening
Cardiomegaly
Nodule Mass
Hernia

